<a href="https://colab.research.google.com/github/bogus1aw/text-classification-benchmark/blob/main/M_herBERT_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HerBERT benchmark for wiki dataset 

In [1]:
# check available GPU
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla T4, 418.67, 15079 MiB


In [2]:
!pip install transformers

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
cuda.is_available()

True

In [4]:
import pandas as pd

def load_corpora_to_dataframe(corpora):
  data = open(corpora).read()
  labels, texts = [], []
  for i, line in enumerate(data.split("\n")):
      content = line.split()
      labels.append(content[0])
      texts.append(" ".join(content[1:]))
  return texts, labels
  # # create a dataframe using texts and lables
  # trainDF = pd.DataFrame()
  # trainDF['text'] = texts
  # trainDF['label'] = labels
  # return trainDF
    
raw_corpora = '/content/drive/MyDrive/master_datasets/wiki_preprocessed/wikiInOneFileDataset.txt'
texts, lables = load_corpora_to_dataframe(raw_corpora)

In [5]:
from sklearn.model_selection import train_test_split
# create test dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, lables, test_size=.2)
# create train and validation dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [6]:
print(len(train_texts), ' ', len(train_labels))
print(len(val_texts), ' ', len(val_labels))
print(len(test_texts), ' ', len(test_labels))

4406   4406
1102   1102
1377   1377


In [15]:
import torch
from transformers import HerbertTokenizer, RobertaForSequenceClassification

tokenizer = HerbertTokenizer.from_pretrained("allegro/herbert-klej-cased-tokenizer-v1")

model = RobertaForSequenceClassification.from_pretrained("allegro/herbert-klej-cased-v1", num_labels=34)

# encoded_input = tokenizer.encode("Kto ma lepszą sztukę, ma lepszy rząd – to jasne.", return_tensors='pt')
# outputs = model(encoded_input)

Some weights of the model checkpoint at allegro/herbert-klej-cased-v1 were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at allegro/herbert-klej-cased-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream

In [8]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=30)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=30)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=30)

from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
train_labels = encoder.fit_transform(train_labels)
val_labels = encoder.fit_transform(val_labels)
test_labels = encoder.fit_transform(test_labels)

In [9]:
# build pyTorch dataset
import torch

class wikiDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = wikiDataset(train_encodings, train_labels)
val_dataset = wikiDataset(val_encodings, val_labels)
test_dataset = wikiDataset(test_encodings, test_labels)

In [10]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()



Step,Training Loss
10,3.562200
20,3.579000
30,3.530200
40,3.558800
50,3.592700
60,3.570700
70,3.556900
80,3.593400
90,3.576000
100,3.497700


TrainOutput(global_step=828, training_loss=3.234023577925088, metrics={'train_runtime': 112.1181, 'train_samples_per_second': 7.385, 'total_flos': 297163121703120, 'epoch': 3.0})

In [12]:
model.predic

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50560, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [11]:
# from torch.utils.data import DataLoader
# from transformers import  AdamW

# # device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')

# model.to(device)
# model.train()

# train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# optim = AdamW(model.parameters(), lr=5e-5)

# for epoch in range(3):
#     for batch in train_loader:
#         optim.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs[0]
#         loss.backward()
#         optim.step()

# model.eval()